In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 

**IMPORT THƯ VIỆN CẦN THIẾT**

In [ ]:
# Import necessary libraries
import json
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset, Dataset


**TẢI VÀ XỬ LÍ DỮ LIỆU**

- Tải dữ liệu từ các tệp JSON (train.json và test.json) chứa ngữ cảnh và câu hỏi.


In [ ]:
# Load the dataset
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

train_data = load_data('/content/train.json')
test_data = load_data('/content/test.json')

- Chuyển đổi dữ liệu thành định dạng Dataset của Hugging Face.

In [ ]:
# Transform the data into HuggingFace Dataset format
def transform_data(data):
    contexts = []
    questions = []
    answers = []

    for item in data:
        context = item['context']
        for qas in item['qas']:
            question = qas['question']
            for answer in qas['answers']:
                contexts.append(context)
                questions.append(question)
                answers.append({
                    'text': answer['text'],
                    'answer_start': answer['answer_start']
                })

    return Dataset.from_dict({
        'context': contexts,
        'question': questions,
        'answers': answers
    })

train_dataset = transform_data(train_data)
test_dataset = transform_data(test_data)


**CHỌN VÀ TINH CHỈNH MÔ HÌNH**

- Sử dụng mô hình deepset/xlm-roberta-base-squad2 được tinh chỉnh cho bài toán hỏi đáp


In [ ]:
# Tokenizer and model
model_checkpoint =  "deepset/xlm-roberta-base-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# Load the model
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

- Chuẩn bị tokenizer từ mô hình đã chọn để mã hóa dữ liệu.
Xử lý dữ liệu để phù hợp với định dạng đầu vào của mô hình.

In [ ]:
# Tokenize the dataset
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]

        if len(answers["text"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"]
            end_char = start_char + len(answers["text"])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)

                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

train_dataset = train_dataset.map(prepare_train_features, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(prepare_train_features, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/50046 [00:00<?, ? examples/s]

Map:   0%|          | 0/15994 [00:00<?, ? examples/s]

**HUẤN LUYỆN MÔ HÌNH**

- Thiết lập các tham số huấn luyện bao gồm đường dẫn lưu kết quả, kích thước batch, số epoch, và sử dụng FP16 để giảm tải bộ nhớ.


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=16,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,  # Sử dụng FP16
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Xóa bộ nhớ cache
torch.cuda.empty_cache()


- Sử dụng Trainer của Hugging Face để huấn luyện mô hình với dữ liệu đã chuẩn bị.

In [ ]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,1.072912
1,1.455800,0.904642
2,1.133500,0.850248


TrainOutput(global_step=1254, training_loss=1.2444229247657497, metrics={'train_runtime': 4969.5611, 'train_samples_per_second': 32.362, 'train_steps_per_second': 0.252, 'total_flos': 3.1455948762906624e+16, 'train_loss': 1.2444229247657497, 'epoch': 2.994179973138338})

**ĐÁNH GIÁ MÔ HÌNH**

In [ ]:
# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.8502483367919922, 'eval_runtime': 144.3028, 'eval_samples_per_second': 118.709, 'eval_steps_per_second': 14.844, 'epoch': 2.994179973138338}


**HÀM DỰ ĐOÁN**

In [ ]:
# Prediction function
def predict(context, question):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)

    # Chuyển inputs sang GPU nếu có
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Chuyển mô hình sang GPU nếu có
    model.to(device)

    outputs = model(**inputs)
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][answer_start:answer_end]))
    return answer

# Test the prediction function
context = "Về mặt kiến ​​trúc, trường có một nhân vật Công giáo. Trên đỉnh mái vòm vàng tòa nhà của chính là bức tượng vàng của Đức Trinh Nữ Maria."
question = "Có gì ngồi trên đầu trang của Tòa nhà Chính tại Notre Dame?"
print(predict(context, question))

bức tượng vàng của Đức Trinh Nữ Maria


**TESTCASE**

In [ ]:
test_cases = [
    {
        "context": "Albert Einstein là một nhà vật lý lý thuyết người Đức đã phát triển thuyết tương đối, một trong hai trụ cột của vật lý hiện đại.",
        "question": "Albert Einstein đã phát triển lý thuyết gì?"
    },
    {
        "context": "Amazon là một công ty công nghệ đa quốc gia của Mỹ có trụ sở tại Seattle, Washington, được thành lập bởi Jeff Bezos vào năm 1994.",
        "question": "Amazon được thành lập vào năm nào?"
    },
    {
        "context": "Công ty Apple Inc. đã được thành lập bởi Steve Jobs, Steve Wozniak và Ronald Wayne vào ngày 1 tháng 4 năm 1976.",
        "question": "Ai là những người sáng lập Apple Inc.?"
    },
    {
        "context": "Đại học Harvard là một trong những trường đại học danh tiếng nhất trên thế giới, được thành lập vào năm 1636 tại Cambridge, Massachusetts.",
        "question": "Đại học Harvard được thành lập vào năm nào?"
    },
    {
        "context": "Python là một ngôn ngữ lập trình bậc cao, thông dịch và hướng đối tượng, được tạo ra bởi Guido van Rossum và ra mắt lần đầu tiên vào năm 1991.",
        "question": "Ngôn ngữ lập trình Python được tạo ra bởi ai?"
    },
    {
        "context": "Điện thoại thông minh đầu tiên, được phát triển bởi IBM, đã được giới thiệu vào năm 1992 và được gọi là Simon Personal Communicator.",
        "question": "Điện thoại thông minh đầu tiên được phát triển bởi công ty nào?"
    },
    {
        "context": "Marie Curie là người phụ nữ đầu tiên đoạt giải Nobel và là người duy nhất đoạt giải Nobel trong hai lĩnh vực khoa học khác nhau.",
        "question": "Marie Curie đã đoạt giải Nobel trong bao nhiêu lĩnh vực khoa học?"
    },
    {
        "context": "Thành phố New York là thành phố đông dân nhất ở Hoa Kỳ, nổi tiếng với Tượng Nữ thần Tự do và Công viên Trung tâm.",
        "question": "Thành phố đông dân nhất ở Hoa Kỳ là thành phố nào?"
    },
    {
        "context": "Mặt trăng là vệ tinh tự nhiên duy nhất của Trái Đất và là thiên thể sáng nhất trên bầu trời đêm.",
        "question": "Vệ tinh tự nhiên duy nhất của Trái Đất là gì?"
    },
    {
        "context": "William Shakespeare là một nhà viết kịch và nhà thơ người Anh, được coi là một trong những nhà văn vĩ đại nhất trong văn học Anh.",
        "question": "William Shakespeare là ai?"
    }
]

for i, case in enumerate(test_cases):
    context = case["context"]
    question = case["question"]
    print(f"Test case {i+1}:")
    print(f"Question: {question}")
    print(f"Predicted answer: {predict(context, question)}")
    print()


Test case 1:
Question: Albert Einstein đã phát triển lý thuyết gì?
Predicted answer: thuyết tương đối,

Test case 2:
Question: Amazon được thành lập vào năm nào?
Predicted answer: 1994.

Test case 3:
Question: Ai là những người sáng lập Apple Inc.?
Predicted answer: Steve Jobs, Steve Wozniak và Ronald Wayne

Test case 4:
Question: Đại học Harvard được thành lập vào năm nào?
Predicted answer: 1636

Test case 5:
Question: Ngôn ngữ lập trình Python được tạo ra bởi ai?
Predicted answer: Guido van Rossum

Test case 6:
Question: Điện thoại thông minh đầu tiên được phát triển bởi công ty nào?
Predicted answer: IBM,

Test case 7:
Question: Marie Curie đã đoạt giải Nobel trong bao nhiêu lĩnh vực khoa học?
Predicted answer: hai

Test case 8:
Question: Thành phố đông dân nhất ở Hoa Kỳ là thành phố nào?
Predicted answer: Thành phố New York

Test case 9:
Question: Vệ tinh tự nhiên duy nhất của Trái Đất là gì?
Predicted answer: Mặt trăng

Test case 10:
Question: William Shakespeare là ai?
Predicted 

In [ ]:
# Định nghĩa các test case với ngữ cảnh dài hơn
long_test_cases = [
    {
        "context": (
            "Albert Einstein là một nhà vật lý lý thuyết người Đức, được coi là một trong những nhà khoa học vĩ đại nhất mọi thời đại. "
            "Ông đã phát triển thuyết tương đối, một trong hai trụ cột của vật lý hiện đại (cùng với cơ học lượng tử). "
            "Lý thuyết của ông đã cách mạng hóa cách chúng ta hiểu về không gian, thời gian và năng lượng. "
            "Einstein đã nhận giải Nobel Vật lý năm 1921 cho những đóng góp của mình cho vật lý lý thuyết, đặc biệt là khám phá ra hiệu ứng quang điện."
        ),
        "question": "Albert Einstein đã nhận giải Nobel Vật lý năm nào?"
    },
    {
        "context": (
            "Amazon là một công ty công nghệ đa quốc gia của Mỹ có trụ sở tại Seattle, Washington. "
            "Công ty được thành lập bởi Jeff Bezos vào ngày 5 tháng 7 năm 1994. "
            "Ban đầu, Amazon bắt đầu như một cửa hàng sách trực tuyến và sau đó mở rộng ra nhiều loại sản phẩm khác nhau. "
            "Ngày nay, Amazon là một trong những công ty lớn nhất thế giới và là nhà bán lẻ trực tuyến lớn nhất toàn cầu. "
            "Amazon cũng cung cấp các dịch vụ đám mây thông qua Amazon Web Services (AWS), dịch vụ phát trực tuyến video và nhạc, và sản xuất thiết bị điện tử."
        ),
        "question": "Amazon được thành lập vào ngày nào?"
    },
    {
        "context": (
            "Steve Jobs, Steve Wozniak và Ronald Wayne là những người đồng sáng lập Apple Inc. vào ngày 1 tháng 4 năm 1976. "
            "Apple bắt đầu với việc sản xuất máy tính cá nhân Apple I và sau đó là Apple II, một trong những máy tính cá nhân đầu tiên thành công về mặt thương mại. "
            "Apple đã cách mạng hóa ngành công nghiệp điện tử tiêu dùng với việc giới thiệu các sản phẩm mang tính biểu tượng như iPhone, iPad, và MacBook. "
            "Steve Jobs được coi là một trong những nhà sáng tạo và doanh nhân vĩ đại nhất trong lịch sử công nghệ."
        ),
        "question": "Apple Inc. được thành lập vào ngày nào?"
    },
    {
        "context": (
            "Google LLC là một công ty công nghệ đa quốc gia của Mỹ chuyên về các dịch vụ và sản phẩm liên quan đến Internet. "
            "Các dịch vụ chính của Google bao gồm công cụ tìm kiếm Google, quảng cáo trực tuyến, điện toán đám mây, phần mềm và phần cứng. "
            "Google được thành lập vào tháng 9 năm 1998 bởi Larry Page và Sergey Brin khi họ còn là sinh viên tiến sĩ tại Đại học Stanford. "
            "Google đã phát triển thành một trong những công ty công nghệ lớn nhất thế giới và có tầm ảnh hưởng rất lớn đến cuộc sống hàng ngày của hàng tỷ người trên toàn cầu."
        ),
        "question": "Google được thành lập vào tháng nào và năm nào?"
    },
    {
        "context": (
            "Microsoft Corporation là một công ty công nghệ đa quốc gia của Mỹ có trụ sở chính tại Redmond, Washington. "
            "Công ty được thành lập bởi Bill Gates và Paul Allen vào ngày 4 tháng 4 năm 1975. "
            "Microsoft phát triển, sản xuất, cấp phép và hỗ trợ một loạt các sản phẩm và dịch vụ phần mềm. "
            "Các sản phẩm nổi tiếng nhất của Microsoft bao gồm hệ điều hành Windows, bộ phần mềm văn phòng Microsoft Office và các thiết bị phần cứng như Xbox và Surface. "
            "Microsoft cũng đầu tư mạnh mẽ vào các lĩnh vực như trí tuệ nhân tạo và điện toán đám mây."
        ),
        "question": "Microsoft được thành lập vào ngày nào?"
    }
]

for i, case in enumerate(long_test_cases):
    context = case["context"]
    question = case["question"]
    print(f"Test case {i+1}:")
    print(f"Question: {question}")
    print(f"Predicted answer: {predict(context, question)}")
    print()

Test case 1:
Question: Albert Einstein đã nhận giải Nobel Vật lý năm nào?
Predicted answer: 1921

Test case 2:
Question: Amazon được thành lập vào ngày nào?
Predicted answer: ngày 5 tháng 7 năm 1994.

Test case 3:
Question: Apple Inc. được thành lập vào ngày nào?
Predicted answer: ngày 1 tháng 4 năm 1976

Test case 4:
Question: Google được thành lập vào tháng nào và năm nào?
Predicted answer: tháng 9 năm 1998

Test case 5:
Question: Microsoft được thành lập vào ngày nào?
Predicted answer: ngày 4 tháng 4 năm 1975



**LƯU MÔ HÌNH**

Mô hình và tokenizer được lưu vào thư mục "trained_model"

In [ ]:
# Lưu mô hình và tokenizer
model.save_pretrained("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/sentencepiece.bpe.model',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

**TẢI LẠI MÔ HÌNH ĐỂ TÁI SỬ DỤNG**

In [ ]:
# Tải lại mô hình và tokenizer để kiểm tra
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

model = AutoModelForQuestionAnswering.from_pretrained("./trained_model")
tokenizer = AutoTokenizer.from_pretrained("./trained_model")